In [ ]:
rootf = FOLDER ## place here the folder where you save PGM, Align_utils and the other files ##

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

curr_float = np.float32
curr_int = np.int16

def convert_number(seqs): # convert to numbers already aligned seqs
    aa = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V',  'W', 'Y','-']
    aadict = {aa[k]: k for k in range(len(aa))}
    msa_num = np.array(list(map(lambda x: [aadict[y] for y in x], seqs[0:])), dtype=curr_int, order="c")
    
    return msa_num

def convert_letter(seqs_n): # convert to numbers already aligned seqs
    aa = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V',  'W', 'Y','-']
    aadictinv = {k: aa[k] for k in range(len(aa))} 
    seqs=[]
    if type(seqs_n[0]) == curr_int:
        seqs.append(''.join([aadictinv[e] for e in seqs_n]))
    else:
        for t in range(len(seqs_n)):
            seqs.append(''.join([aadictinv[e] for e in seqs_n[t]]))
    return seqs

#%matplotlib inline
import sys, os, pickle
sys.path.append(rootf + '/PGM/source/')
sys.path.append(rootf + '/PGM/utilities/')
sys.path.append(rootf + '/Align_utils/')
from common_imports import set_num_threads
set_num_threads(1) # Set the number of cores. Must be executed before importing numpy&numba.
import rbm,utilities
import Proteins_utils, RBM_utils, utilities, sequence_logo, plots_utils

In [ ]:
## read tables with counts before and after antigen stimulation ##
table_data = pd.read_csv(rootf + '/example_file.tsv', sep='\t', low_memory=False)
        
seqs = list(table_data['CDR3'])
mult_seqs_post = table_data['Count post-stimulation'].values
mult_seqs_pre = table_data['Count pre-stimulation'].values

In [ ]:
## This block is to align the sequences##

import subprocess
name_mat = rootf + '/Align_utils/align_protpy.py'

SA = 19
SAmin = 5
SAmax = 23

all_seqs1 = seqs

name_seed = rootf + '/Align_utils/prots_seed.txt'
with open(name_seed, 'w') as out_f:
    for u in range(len(all_seqs1)):
        if len(all_seqs1[u]) >= 5:  
            out_f.write(all_seqs1[u] + '\n')

subprocess.call('python3 ' + name_mat + ' -ss ' + name_seed + ' -SA ' + str(SA) + ' -SAmin ' + str(SAmin) + ' -SAmax ' + str(SAmax), shell = True)

seqs_al = np.loadtxt(rootf + '/Align_utils/aligned_prot.txt')
sn = seqs_al.astype(np.int)
seqs_al = convert_letter(sn) ## this is the aligned dataset ##

In [ ]:
## produce count weighted datasets, i.e. datasets where each sequence is present as many times as its count pre or post stimulation ##
import numpy.matlib 
import random

seqs_post_training = []
for u in range(len(seqs_al)):
    repl = np.matlib.repmat(seqs_al[u], mult_seqs_post[u], 1).tolist()
    for y in repl:
        seqs_post_training.append(y[0]) 
        
seqs_pre_training = []
for u in range(len(seqs_al)):
    repl = np.matlib.repmat(seqs_al[u], mult_seqs_pre[u], 1).tolist()
    for y in repl:
        seqs_pre_training.append(y[0]) 

random.shuffle(seqs_post_training)
random.shuffle(seqs_pre_training)

filename = rootf + '/example_file_post.fasta'
sequences = seqs_post_training

all_labels = ['S%s'%k for k in range(len(sequences))]
with open(filename,'w') as fil:
    for seq, label in zip(sequences,all_labels):
        fil.write('>%s\n'%label)
        fil.write('%s\n'%seq)
        
filename = rootf + '/example_file_pre.fasta'
sequences = seqs_pre_training

all_labels = ['S%s'%k for k in range(len(sequences))]
with open(filename,'w') as fil:
    for seq, label in zip(sequences,all_labels):
        fil.write('>%s\n'%label)
        fil.write('%s\n'%seq)

In [ ]:
## Here I select load the sample in FASTA format I have just produced ##

path_in = rootf  ## simply the folder where the CDR3 sample is saved - note it is a count-weighted dataset of aligned sequences  

TS=80 ## percentage of the dataset to learn the model ##

# Load Data
filename_post = '/example_file_post.fasta'
all_data_post = Proteins_utils.load_FASTA(path_in + filename_post,with_labels=False,remove_insertions=False, drop_duplicates=False)

filename_pre = '/example_file_pre.fasta'
all_data_pre = Proteins_utils.load_FASTA(path_in + filename_pre,with_labels=False,remove_insertions=False, drop_duplicates=False)

## From here, I learn RBM and next manipulate the outcome ##
n_v = 19 # Number of visible units; = # sites in alignment.
n_h = 25 # Number of hidden units.
l12 = 0.1

# Decide the name for the output #
namem_pre = '/model_' + str(n_h) + '_' + str(l12) + '_pre.data'
namem_post = '/model_' + str(n_h) + '_' + str(l12) + '_post.data'

## RBM training ##
maketraining = 1 # Variable to decide whether you want to train a new model or to load an existing one 

if maketraining:
    
    ## These are parameters for the training ##
    visible = 'Potts' # Nature of visible units potential. Here, Potts states...
    n_cv = 21 # With n_cv = 21 colors (all possible amino acids + gap)
    hidden = 'dReLU' # Nature of hidden units potential. Here, dReLU potential.
    # hidden = 'Gaussian' # Nature of hidden units potential. Here, dReLU potential.
    seed = 0 # Random seed (optional)
    batch_size = 100 # Size of mini-batches (and number of Markov chains used). Default: 100.
    n_iter = 200 # Number of epochs
    learning_rate = 0.1 # Initial learning rate (default: 0.1)
    decay_after = 0.5 # Decay learning rate after 50% of iterations (default: 0.5)	
    N_MC = 10 # Number of Monte Carlo steps between each update
    l1b=l12
    
    B = all_data_post.shape[0]
    RBM_post = rbm.RBM(visible = visible,hidden = hidden ,n_v = n_v,n_h = n_h, n_cv = n_cv, random_state = seed, zero_field = False)
    l2f = 1/len(all_data_post[0:int(B/100*TS)])
    RBM_post.fit(all_data_post[0:int(B/100*TS)], weights=None, batch_size = batch_size,n_iter = n_iter, l1b = l1b, l2_fields = l2f, N_MC = N_MC, decay_after = decay_after, verbose = 1)
    RBM_utils.saveRBM(rootf + namem_post, RBM_post)
    
    B = all_data_pre.shape[0]
    RBM_pre = rbm.RBM(visible = visible,hidden = hidden,n_v = n_v,n_h = n_h, n_cv = n_cv, random_state = seed, zero_field = False)
    l2f = 1/len(all_data_pre[0:int(B/100*TS)])
    RBM_pre.fit(all_data_pre[0:int(B/100*TS)], weights=None, batch_size = batch_size,n_iter = n_iter, l1b = l1b, l2_fields = l2f, N_MC = N_MC, decay_after = decay_after, verbose = 1)
    RBM_utils.saveRBM(rootf + namem_pre, RBM_pre)
    
else:

    RBM_pre = RBM_utils.loadRBM(namem_pre)
    RBM_post = RBM_utils.loadRBM(namem_post)

In [ ]:
## Model's parameters inspection ##

weights = RBM_post.weights ## Interesting features are the ones picked up in the 3 weeks sample ##

s2=16;
interesting_features = [0,2,6,7,11,13,17] ## select what hidden units inspect 
for i in range(len(interesting_features)):
    fig = sequence_logo.Sequence_logo(weights[interesting_features[i]], figsize=(5.5,1.8), ylabel = 'Weights ' + str(interesting_features[i]+1),  ticks_every=5, ticks_labels_size=s2-2) 

In [ ]:
## Evaluate the model on the data##
seqs_n = convert_number(seqs_al)

ll_post = RBM_post.likelihood(seqs_n)
ll_pre = RBM_pre.likelihood(seqs_n)

## Projection onto hidden units space ##

ind_sort = np.argsort(ll_post)[::-1]
ntop = 10 ## how many sequenes to plot##

I = RBM_post.vlayer.compute_output(seqs_n[ind_sort[:ntop]],RBM_post.weights)

ix = 11
iy = 10

s1 = 120.0
fig, ax = plt.subplots()
fig.set_figwidth(6.2)
fig.set_figheight(6)

listan = [seqs[ind_sort[t]] for t in range(ntop)]
x=I[:,ix]
y=I[:,iy]

sc = ax.scatter(x,y, s=s1, c = np.log10(mult_seqs_post[ind_sort[:ntop]]), edgecolors='k', cmap='YlGn')
for i, txt in enumerate(listan):
    ax.annotate(txt, (x[i]-1, y[i]+0.25), fontfamily='monospace', fontvariant='small-caps', fontsize=14)

s2=18
ax.set_xlabel('Input to ' + str(ix+1), fontsize=s2)
ax.set_ylabel('Input to ' + str(iy+1), fontsize=s2)

cbar = plt.colorbar(mappable=sc, ax=ax, shrink=0.7,orientation='horizontal')
cbar.ax.tick_params(labelsize=s2)

ax.tick_params(axis='both', which='major', labelsize = s2)
ax.yaxis.set_label_position("right")
ax.yaxis.tick_right()
plt.tight_layout()

In [ ]:
ind_sortm = np.argsort(mult_seqs_post)[::-1] 

## The likelihood is correlated to the counts ##
print('Correlation to count - likelihood of 100 most abundant clones')
print(np.corrcoef(ll_post[ind_sortm[:100]], np.log10(mult_seqs_post[ind_sortm[:100]]+1/2))[0,1])

## Here I show the correlation of the response score to the fold change ##
print('Correlation to fold change - response score of 100 most abundant clones')
print(np.corrcoef(ll_post[ind_sortm[:100]] - ll_pre[ind_sortm[:100]], np.log10(mult_seqs_post[ind_sortm[:100]]+1/2) - np.log10(mult_seqs_pre[ind_sortm[:100]]+1/2))[0,1])